In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print("using", device, "device")


using cuda device


In [3]:
import tensorflow

2023-08-21 21:52:24.472054: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-21 21:52:24.497646: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 21:52:24.894716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

ModuleNotFoundError: No module named 'auto_gptq'

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

"""
To download from a specific branch, use the revision parameter, as in this example:

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        revision="gptq-4bit-32g-actorder_True",
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        quantize_config=None)
"""

prompt = "Tell me about AI"
system_message = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

print(pipe(prompt_template)[0]['generated_text'])


In [4]:
pip install safetensors

/bin/bash: /home/ivib/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Obtaining dependency information for safetensors from https://files.pythonhosted.org/packages/61/57/402c6a522f26e6bdc8d46cce379bf20f40daa0764578510e4e8a3d3ed1a9/safetensors-0.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
from safetensors import safe_open
from safetensors.torch import save_file

tensors = {
   "weight1": torch.zeros((1024, 1024)),
   "weight2": torch.zeros((1024, 1024))
}
save_file(tensors, "model.safetensors")

tensors = {}
with safe_open("model.safetensors", framework="pt", device="cpu") as f:
   for key in f.keys():
       tensors[key] = f.get_tensor(key)

In [11]:
import torch
from safetensors import safe_open
from safetensors.torch import save_file

tensors = {}
with safe_open("C:\Users\vibud\Desktop\Github\LLM-Chatbots\LLMs\TheBloke_Llama-2-13B-chat-GPTQ\gptq_model-4bit-128g.safetensors", 
               framework="pt", 
               device="cpu") as f:
   for key in f.keys():
       tensors[key] = f.get_tensor(key)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2597160081.py, line 6)

In [14]:
AutoModelForCausalLM.from_pretrained("C:\\Users\\vibud\\Desktop\\Github\\LLM-Chatbots\\LLMs\\TheBloke_Llama-2-13B-chat-GPTQ\\", 
                                    use_safetensors=True)

NameError: name 'AutoModelForCausalLM' is not defined

In [16]:
pip install transformers

/bin/bash: /home/ivib/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/21/02/ae8e595f45b6c8edee07913892b3b41f5f5f273962ad98851dc6a564bbb9/transformers-4.31.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 902.4 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for huggingface-hub<1.0,>=0.14.1 from https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.16.4-py3-none-any.whl.metadata (12 kB)
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/c0/f4/278e305e02245937579a7952b8a3205116b4d2480a3c03fa11e599b773d6/regex-2023.8.8-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer

AutoModelForCausalLM.from_pretrained("C:\\Users\\vibud\\Desktop\\Github\\LLM-Chatbots\\LLMs\\TheBloke_Llama-2-13B-chat-GPTQ\\", 
                                    use_safetensors=True)

2023-08-21 19:15:08.663008: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-21 19:15:08.804130: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 19:15:09.451901: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


OSError: Can't load the configuration of 'C:\Users\vibud\Desktop\Github\LLM-Chatbots\LLMs\TheBloke_Llama-2-13B-chat-GPTQ\'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'C:\Users\vibud\Desktop\Github\LLM-Chatbots\LLMs\TheBloke_Llama-2-13B-chat-GPTQ\' is the correct path to a directory containing a config.json file

In [18]:
pip install accelerate

/bin/bash: /home/ivib/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/70/f9/c381bcdd0c3829d723aa14eec8e75c6c377b4ca61ec68b8093d9f35fc7a7/accelerate-0.21.0-py3-none-any.whl.metadata
  Using cached accelerate-0.21.0-py3-none-any.whl.metadata (17 kB)
Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)
Note: you may need to restart the kernel to use updated packages.


In [19]:
# %pip install transformers
# %pip install accelerate
!pip install huggingface-hub==0.14.1

!huggingface-cli login --token "my_token"

from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

/bin/bash: /home/ivib/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4
/bin/bash: /home/ivib/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Traceback (most recent call last):
  File "/home/ivib/miniconda3/envs/tf/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/home/ivib/miniconda3/envs/tf/lib/python3.9/site-packages/huggingface_hub/commands/huggingface_cli.py", line 45, in main
    service.run()
  File "/home/ivib/miniconda3/envs/tf/lib/python3.9/site-packages/hug

OSError: meta-llama/Llama-2-7b-chat-hf is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [3]:
import time

matrix_size = 32*512

x = torch.randn(matrix_size, matrix_size)
y = torch.randn(matrix_size, matrix_size)

print("************ CPU SPEED ***************")
start = time.time()
result = torch.matmul(x,y)
print(time.time() - start)
print("verify device:", result.device)

x_gpu = x.to(device)
y_gpu = y.to(device)
torch.cuda.synchronize()

for i in range(3):
    print("************ GPU SPEED ***************")
    start = time.time()
    result_gpu = torch.matmul(x_gpu,y_gpu)
    torch.cuda.synchronize()
    print(time.time() - start)
    print(str(i) , ": verify device:", result_gpu.device)

************ CPU SPEED ***************
9.959770679473877
verify device: cpu
************ GPU SPEED ***************
0.9555909633636475
0 : verify device: cuda:0
************ GPU SPEED ***************
0.3858306407928467
1 : verify device: cuda:0
************ GPU SPEED ***************
0.35582733154296875
2 : verify device: cuda:0


In [ ]:
import time

matrix_size = 32*512

x = torch.randn(matrix_size, matrix_size)
y = torch.randn(matrix_size, matrix_size)

print("************ CPU SPEED ***************")
start = time.time()
result = torch.matmul(x,y)
print(time.time() - start)
print("verify device:", result.device)

x_gpu = x.to(device)
y_gpu = y.to(device)
torch.cuda.synchronize()

for i in range(1000):
    print("************ GPU SPEED ***************")
    start = time.time()
    result_gpu = torch.matmul(x_gpu,y_gpu)
    torch.cuda.synchronize()
    print(time.time() - start)
    print(str(i) , ": verify device:", result_gpu.device)

************ CPU SPEED ***************
9.998383522033691
verify device: cpu
************ GPU SPEED ***************
0.6521763801574707
0 : verify device: cuda:0
************ GPU SPEED ***************
0.5582013130187988
1 : verify device: cuda:0
************ GPU SPEED ***************
0.47942090034484863
2 : verify device: cuda:0
************ GPU SPEED ***************
0.4291203022003174
3 : verify device: cuda:0
************ GPU SPEED ***************
0.38628602027893066
4 : verify device: cuda:0
************ GPU SPEED ***************
0.3582878112792969
5 : verify device: cuda:0
************ GPU SPEED ***************
0.3561286926269531
6 : verify device: cuda:0
************ GPU SPEED ***************
0.3635852336883545
7 : verify device: cuda:0
************ GPU SPEED ***************
0.361407995223999
8 : verify device: cuda:0
************ GPU SPEED ***************
0.35755491256713867
9 : verify device: cuda:0
************ GPU SPEED ***************
0.3655681610107422
10 : verify device: cuda

0.38489246368408203
190 : verify device: cuda:0
************ GPU SPEED ***************
0.3872413635253906
191 : verify device: cuda:0
************ GPU SPEED ***************
0.3865971565246582
192 : verify device: cuda:0
************ GPU SPEED ***************
0.3853428363800049
193 : verify device: cuda:0
************ GPU SPEED ***************
0.3879814147949219
194 : verify device: cuda:0
************ GPU SPEED ***************
0.38549017906188965
195 : verify device: cuda:0
************ GPU SPEED ***************
0.3872835636138916
196 : verify device: cuda:0
************ GPU SPEED ***************
0.38881993293762207
197 : verify device: cuda:0
************ GPU SPEED ***************
0.38408398628234863
198 : verify device: cuda:0
************ GPU SPEED ***************
0.38849616050720215
199 : verify device: cuda:0
************ GPU SPEED ***************
0.38835573196411133
200 : verify device: cuda:0
************ GPU SPEED ***************
0.38611292839050293
201 : verify device: cuda:0


0.39032483100891113
285 : verify device: cuda:0
************ GPU SPEED ***************
0.3929910659790039
286 : verify device: cuda:0
************ GPU SPEED ***************
0.3897082805633545
287 : verify device: cuda:0
************ GPU SPEED ***************
0.391923189163208
288 : verify device: cuda:0
************ GPU SPEED ***************
0.39280128479003906
289 : verify device: cuda:0
************ GPU SPEED ***************
0.3891749382019043
290 : verify device: cuda:0
************ GPU SPEED ***************
0.39139342308044434
291 : verify device: cuda:0
************ GPU SPEED ***************
0.39104723930358887
292 : verify device: cuda:0
************ GPU SPEED ***************
0.38785862922668457
293 : verify device: cuda:0
************ GPU SPEED ***************
0.3937854766845703
294 : verify device: cuda:0
************ GPU SPEED ***************
0.3922293186187744
295 : verify device: cuda:0
************ GPU SPEED ***************
0.39351487159729004
296 : verify device: cuda:0
**

0.3926384449005127
381 : verify device: cuda:0
************ GPU SPEED ***************
0.38994646072387695
382 : verify device: cuda:0
************ GPU SPEED ***************
0.3924715518951416
383 : verify device: cuda:0
************ GPU SPEED ***************
0.3935983180999756
384 : verify device: cuda:0
************ GPU SPEED ***************
0.3903181552886963
385 : verify device: cuda:0
************ GPU SPEED ***************
0.3938431739807129
386 : verify device: cuda:0
************ GPU SPEED ***************
0.39290618896484375
387 : verify device: cuda:0
************ GPU SPEED ***************
0.38941335678100586
388 : verify device: cuda:0
************ GPU SPEED ***************
0.39449334144592285
389 : verify device: cuda:0
************ GPU SPEED ***************
0.3908047676086426
390 : verify device: cuda:0
************ GPU SPEED ***************
0.39173102378845215
391 : verify device: cuda:0
************ GPU SPEED ***************
0.3931145668029785
392 : verify device: cuda:0
**

0.3905022144317627
476 : verify device: cuda:0
************ GPU SPEED ***************
0.3918592929840088
477 : verify device: cuda:0
************ GPU SPEED ***************
0.39614367485046387
478 : verify device: cuda:0
************ GPU SPEED ***************
0.39028191566467285
479 : verify device: cuda:0
************ GPU SPEED ***************
0.3913273811340332
480 : verify device: cuda:0
************ GPU SPEED ***************
0.3923676013946533
481 : verify device: cuda:0
************ GPU SPEED ***************
0.38850903511047363
482 : verify device: cuda:0
************ GPU SPEED ***************
0.39244771003723145
483 : verify device: cuda:0
************ GPU SPEED ***************
0.3905456066131592
484 : verify device: cuda:0
************ GPU SPEED ***************
0.38883495330810547
485 : verify device: cuda:0
************ GPU SPEED ***************
0.39092135429382324
486 : verify device: cuda:0
************ GPU SPEED ***************
0.38775134086608887
487 : verify device: cuda:0


0.3948640823364258
571 : verify device: cuda:0
************ GPU SPEED ***************
0.39862585067749023
572 : verify device: cuda:0
************ GPU SPEED ***************
0.3973531723022461
573 : verify device: cuda:0
************ GPU SPEED ***************
0.3967418670654297
574 : verify device: cuda:0
************ GPU SPEED ***************
0.3962280750274658
575 : verify device: cuda:0
************ GPU SPEED ***************
0.393449068069458
576 : verify device: cuda:0
************ GPU SPEED ***************
0.3922717571258545
577 : verify device: cuda:0
************ GPU SPEED ***************
0.39424824714660645
578 : verify device: cuda:0
************ GPU SPEED ***************
0.39141011238098145
579 : verify device: cuda:0
************ GPU SPEED ***************
0.3908967971801758
580 : verify device: cuda:0
************ GPU SPEED ***************
0.39141082763671875
581 : verify device: cuda:0
************ GPU SPEED ***************
0.38745999336242676
582 : verify device: cuda:0
***

0.39603281021118164
666 : verify device: cuda:0
************ GPU SPEED ***************
0.38993358612060547
667 : verify device: cuda:0
************ GPU SPEED ***************
0.3889312744140625
668 : verify device: cuda:0
************ GPU SPEED ***************
0.39638543128967285
669 : verify device: cuda:0
************ GPU SPEED ***************
0.3911018371582031
670 : verify device: cuda:0
************ GPU SPEED ***************
0.396895170211792
671 : verify device: cuda:0
************ GPU SPEED ***************
0.3875844478607178
672 : verify device: cuda:0
************ GPU SPEED ***************
0.39801597595214844
673 : verify device: cuda:0
************ GPU SPEED ***************
0.39560818672180176
674 : verify device: cuda:0
************ GPU SPEED ***************
0.396425724029541
675 : verify device: cuda:0
************ GPU SPEED ***************
0.39052319526672363
676 : verify device: cuda:0
************ GPU SPEED ***************
0.387829065322876
677 : verify device: cuda:0
****

0.3922696113586426
762 : verify device: cuda:0
************ GPU SPEED ***************
0.38856077194213867
763 : verify device: cuda:0
************ GPU SPEED ***************
0.38979053497314453
764 : verify device: cuda:0
************ GPU SPEED ***************
0.39288759231567383
765 : verify device: cuda:0
************ GPU SPEED ***************
0.3896951675415039
766 : verify device: cuda:0
************ GPU SPEED ***************
0.3895704746246338
767 : verify device: cuda:0
************ GPU SPEED ***************
0.3890674114227295
768 : verify device: cuda:0
************ GPU SPEED ***************
0.3904876708984375
769 : verify device: cuda:0
************ GPU SPEED ***************
0.38939380645751953
770 : verify device: cuda:0
************ GPU SPEED ***************
0.3881683349609375
771 : verify device: cuda:0
************ GPU SPEED ***************
0.3910951614379883
772 : verify device: cuda:0
************ GPU SPEED ***************
0.3895699977874756
773 : verify device: cuda:0
***

In [ ]:
matrix_size = 32*512

for i in range(100):
    x = torch.randn(matrix_size, matrix_size)
    y = torch.randn(matrix_size, matrix_size)

    print("************ CPU SPEED ***************")
    start = time.time()
    result = torch.matmul(x,y)
    print(time.time() - start)
    print("verify device:", result.device)

    x_gpu = x.to(device)
    y_gpu = y.to(device)
    torch.cuda.synchronize()    
    
    print("************ GPU SPEED ***************")
    start = time.time()
    result_gpu = torch.matmul(x_gpu,y_gpu)
    torch.cuda.synchronize()
    print(time.time() - start)
    print("verify device:", result_gpu.device)